### List of all products 

In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import json
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [2]:
import requests

BASE_URL='https://www.openfood.ch/api/v3'
KEY='97bb347d26f3a8b39b4558827c3fcf56'
ENDPOINT='/products'



url = BASE_URL + ENDPOINT

headers = {
  'Authorization': 'Token token=' + KEY,
  'Accept': 'application/json',
  'Content-Type': 'application/vnd.api+json',
  'Accept-Encoding': 'gzip,deflate'
}
productlist = []
listlength = 0
i = 0
for i in range(0, 25):
    query = {
        'page[number]': i,
        'page[size]': 200  # maximum
    }
    r = requests.get(url, params=query, headers=headers)
    if r.status_code == 200:
        print('Page ' + str(i) + ' loaded successfully.')

        for product in r.json()['data']:
            try:
                productlist.append(product['id'])
            except:
                pass

    if listlength == len(productlist):
        break
    #print('Product count: ' + str(len(productlist)))
    listlength = len(productlist)
    i += 1
    
'''
url = BASE_URL + ENDPOINT

query = {
  'page[number]': 1,
  'page[size]': 200
}

headers = {
  'Authorization': 'Token token=' + KEY,
  'Accept': 'application/json',
  'Content-Type': 'application/vnd.api+json',
  'Accept-Encoding': 'gzip,deflate'
}

r = requests.get(url, params=query, headers=headers)
print("Status: " + str(r.status_code))
if r.status_code == 200:
    print('Page loaded successfully.')
    print('Generated in ' + str(r.json().get('meta', {}).get('generated_in', -1)) + ' milliseconds.')
    print('Next page\'s URL is: ' + r.json()['links'].get('next', 'UNKNOWN'))
    print('Barcodes on this page:')
    for product in r.json()['data']:
        productlist = []
        for x in r.json()['data']:
            productlist.append(x['id'])
print(productlist)
'''

Page 0 loaded successfully.
Page 1 loaded successfully.
Page 2 loaded successfully.
Page 3 loaded successfully.
Page 4 loaded successfully.
Page 5 loaded successfully.
Page 6 loaded successfully.
Page 7 loaded successfully.
Page 8 loaded successfully.
Page 9 loaded successfully.
Page 10 loaded successfully.
Page 11 loaded successfully.
Page 12 loaded successfully.
Page 13 loaded successfully.
Page 14 loaded successfully.
Page 15 loaded successfully.
Page 16 loaded successfully.
Page 17 loaded successfully.
Page 18 loaded successfully.
Page 19 loaded successfully.
Page 20 loaded successfully.
Page 21 loaded successfully.
Page 22 loaded successfully.
Page 23 loaded successfully.
Page 24 loaded successfully.


'\nurl = BASE_URL + ENDPOINT\n\nquery = {\n  \'page[number]\': 1,\n  \'page[size]\': 200\n}\n\nheaders = {\n  \'Authorization\': \'Token token=\' + KEY,\n  \'Accept\': \'application/json\',\n  \'Content-Type\': \'application/vnd.api+json\',\n  \'Accept-Encoding\': \'gzip,deflate\'\n}\n\nr = requests.get(url, params=query, headers=headers)\nprint("Status: " + str(r.status_code))\nif r.status_code == 200:\n    print(\'Page loaded successfully.\')\n    print(\'Generated in \' + str(r.json().get(\'meta\', {}).get(\'generated_in\', -1)) + \' milliseconds.\')\n    print(\'Next page\'s URL is: \' + r.json()[\'links\'].get(\'next\', \'UNKNOWN\'))\n    print(\'Barcodes on this page:\')\n    for product in r.json()[\'data\']:\n        productlist = []\n        for x in r.json()[\'data\']:\n            productlist.append(x[\'id\'])\nprint(productlist)\n'

### Getting list of useful characteristics from id

In [3]:
%%time
BASE_URL='https://www.openfood.ch/api/v3'
KEY='97bb347d26f3a8b39b4558827c3fcf56'
dInfo = {}
dIngredients = {}
dNutrients = {}

for PRODUCT_ID in productlist:
    ENDPOINT='/products/{}'.format(PRODUCT_ID)

    url = BASE_URL + ENDPOINT

    headers = {
      'Authorization': 'Token token=' + KEY,
      'Accept': 'application/json',
      'Content-Type': 'application/vnd.api+json',
      'Accept-Encoding': 'gzip,deflate'
    }

    r = requests.get(url, headers=headers)
    #print ('Status: ' + str(r.status_code))
    item = {}
    item_ing = {}
    item_nut = {}
    data = r.json()['data']
    if r.status_code == 200:
        try:
            item['name'] = item_ing['name'] = item_nut['name'] = data['display_name_translations']['fr']
            item['id'] = item_ing['id'] = item_nut['id'] = data['id']
            #item['barcode'] = item_ing['barcode'] = item_nut['barcode'] = data['barcode']
            
            item['quantity'] = data['quantity']
            item['unit'] = data['unit']
            item['portion'] = data['portion_quantity']
            item['portion unit'] = data['portion_unit']
            item['alcohol'] = data['alcohol_by_volume']
            
            item_ing['ingredients'] = data['ingredients_translations']['fr']
            
            nuts = {}
            for nut in data['nutrients'].keys():
                nuts[nut] = {'per_day':data['nutrients'][nut]['per_day'],
                             'per_hundred':data['nutrients'][nut]['per_hundred'],
                             'per_portion':data['nutrients'][nut]['per_portion']}
            item_nut['nutrients'] = nuts
            
            dInfo[data['barcode']] = item    
            dIngredients[data['barcode']] = item_ing
            dNutrients[data['barcode']] = item_nut
        except:
            pass

CPU times: user 3min 57s, sys: 7.45 s, total: 4min 4s
Wall time: 19min 20s


### Translate ingredients

In [4]:
%%time
from watson_developer_cloud import LanguageTranslatorV2 as LanguageTranslator

language_translator = LanguageTranslator(username='a69f77dc-9248-4069-85b5-1ac1ef6bfd7d',password='UQznP8xcKtuq')

count = 0
remove = []
for barcode, tmp in dIngredients.items():
    try:        
        ingredients_ = ' '.join(tmp['ingredients'][1].splitlines())
        language_json = language_translator.identify(ingredients_)
        language = pd.DataFrame(language_json['languages']).sort_values(by=['confidence'], ascending=False)
        source_lang = language.iloc[0, 1]
        if source_lang != "en":
            translation = language_translator.translate(text=ingredients_, source=source_lang, target='en')
            dIngredients[barcode]['ingredients'] = "en_translated, " + json.dumps(translation, indent=2, ensure_ascii=False)
        else:
            dIngredients[barcode]['ingredients'] = "en_kept, " + ingredients_       
        name_ = ' '.join(tmp['name'][1].splitlines())
        language_json = language_translator.identify(name_)
        language = pd.DataFrame(language_json['languages']).sort_values(by=['confidence'], ascending=False)
        source_lang = language.iloc[0, 1]
        if source_lang != "en":
            translation = language_translator.translate(text=ingredients_, source=source_lang, target='en')
            dInfo[barcode]['name'] = "en_translated, " + json.dumps(translation, indent=2, ensure_ascii=False)
            dIngredients[barcode]['name'] = "en_translated, " + json.dumps(translation, indent=2, ensure_ascii=False)
            dNutrients[barcode]['name'] = "en_translated, " + json.dumps(translation, indent=2, ensure_ascii=False)
        else:
            dInfo[barcode]['name'] = "en_kept, " + name_
            dIngredients[barcode]['name'] = "en_kept, " + name_
            dNutrients[barcode]['name'] = "en_kept, " + name_
                        
    except:
        remove.append(barcode)
        
    count += 1
    if count % 50 == 0:
        print("%s/%s items done!"%(count, len(dIngredients)))

# remove the ones (from everywhere) which couldn't be translated
for barcode in remove:
    del dInfo[barcode]
    del dIngredients[barcode]
    del dNutrients[barcode]

50/4224 items done!
100/4224 items done!
150/4224 items done!
200/4224 items done!
250/4224 items done!
300/4224 items done!
350/4224 items done!
400/4224 items done!
450/4224 items done!
500/4224 items done!
550/4224 items done!
600/4224 items done!
650/4224 items done!
700/4224 items done!
750/4224 items done!
800/4224 items done!
850/4224 items done!
900/4224 items done!
950/4224 items done!
1000/4224 items done!
1050/4224 items done!
1100/4224 items done!
1150/4224 items done!
1200/4224 items done!
1250/4224 items done!
1300/4224 items done!
1350/4224 items done!
1400/4224 items done!
1450/4224 items done!
1500/4224 items done!
1550/4224 items done!
1600/4224 items done!
1650/4224 items done!
1700/4224 items done!
1750/4224 items done!
1800/4224 items done!
1850/4224 items done!
1900/4224 items done!
1950/4224 items done!
2000/4224 items done!
2050/4224 items done!
2100/4224 items done!
2150/4224 items done!
2200/4224 items done!
2250/4224 items done!
2300/4224 items done!
2350/422

In [5]:
# dump dictionaries to json files (load it in with `json.load(fName)`)

with open("dInfo.json", "w") as f_:
    json.dump(dInfo, f_)
with open("dIngredients.json", "w") as f_:
    json.dump(dIngredients, f_)
with open("dNutrients.json", "w") as f_:
    json.dump(dNutrients, f_)

### Calculate climate impact!

In [1]:
import json

with open("dInfo.json", "r") as f_:
    dInfo = json.load(f_)
with open("dIngredients.json", "r") as f_:
    dIngredients = json.load(f_)
with open("dNutrients.json", "r") as f_:
    dNutrients = json.load(f_)

In [2]:
from food_db import build_database
ingredients_to_impact = build_database()

In [3]:
import re
ingredient_dict = {}

for item in list(dIngredients.items()):
    lItems = item[1]['ingredients'].split(',')[1:]
    quantity_dict = {}
    i = 1
    estimate = 50
    for l in lItems:
        percentage = re.findall(r'\d+.\d+%', l)
        if(len(percentage) == 0):
            quantity_dict[l] = estimate
        else:
            quantity_dict[l] = percentage[0].rstrip('%')
        i = i + 1
        estimate = estimate/2
    ingredient_dict[item[0]] = quantity_dict

In [4]:
final_dict = {}
for bar_code in list(ingredient_dict):
    temp_dict = {}
    temp_dict['ingredients'] = ingredient_dict[bar_code]
    temp_dict['quantity'] = dInfo[bar_code]['quantity']
    temp_dict['unit'] = dInfo[bar_code]['unit']
    final_dict[bar_code] = temp_dict
    

In [5]:
import difflib
footprint_dict = {}

for item in final_dict.items():
    footprint = 0
    ingredients = item[1]['ingredients']
    quantity = float(final_dict[item[0]]['quantity'])
    for ing_key, ing_quantity in ingredients.items():
        print(ing_key)
        scores = []
        keys = []
        for key, value in ingredients_to_impact.items():
            keys.append(key)
            scores.append(difflib.SequenceMatcher(None, ing_key,key).ratio())
        matched_score = ingredients_to_impact[keys[scores.index(max(scores))]]
        print(quantity,ing_quantity,matched_score)
        footprint += float(quantity)*float(ing_quantity)*0.01*float(matched_score)
    footprint_dict[item[0]] = footprint

 garlic
0.0 0.1953125 0.04
 potatoes
0.0 1.5625 0.33
 salt
0.0 0.09765625 0.22
 wheat flour
0.0 6.25 0.3
 water
0.0 0.390625 0.44
 modified cornstarch
0.0 0.01220703125 4.51
 "Paste (water
0.0 50 1.24
 salt)
0.0 0.0244140625 0.22
 5% tofu (soybeans
0.0 0.048828125 0.56
 soya beans
0.0 0.78125 0.56
 spices
0.0 0.0030517578125 0.34
 dye: caramel
0.0 0.006103515625 1.82
 preservative: E 202 ."
0.0 0.00152587890625 0.89
 sugar
0.0 3.125 3.01
 salt
0.0 0.1953125 0.22
 water
0.0 25.0 0.44
 cumin
0.0 0.0003814697265625 0.14
 garlic puree
0.0 4.76837158203125e-05 0.04
 preservative: E 220)
0.0 1.5625 0.89
 mint
0.0 0.09765625 0.14
 cinnamon
0.0 0.006103515625 2.88
 apricots 6% (with rice flour
0.0 3.125 0.23
 coriander leaves
0.0 2.384185791015625e-05 2.41
 cayenne pepper
0.0 0.01220703125 0.26
 "Tomatoes
0.0 50 0.23
 spice (1.5% rice flour
0.0 1.5 1.82
 acid
0.0 1.1920928955078125e-05 5.25
 chili powder
0.0 0.0244140625 0.3
 parsley
0.0 0.000762939453125 0.15
 coriander
0.0 0.390625 0.12
 cit

100.0 0.000762939453125 0.04
 salt
100.0 0.09765625 0.22
 oregano
100.0 0.0244140625 0.12
 cumin
100.0 9.5367431640625e-05 0.14
 mint
100.0 4.76837158203125e-05 0.14
 rapeseed
100.0 5.9604644775390625e-06 0.89
 seasoning (salt
100.0 25.0 1.48
 mustard
100.0 3.125 1.0
 chili powder
100.0 0.78125 0.3
 chilli 7%
100.0 0.390625 3.43
?
100.0 0.00152587890625 0.22
 coriander
100.0 6.25 0.12
 "Water
100.0 50 0.44
?Antiklumpmittel: E551 flavors)
100.0 1.1920928955078125e-05 1.53
 peppers
100.0 0.0003814697265625 0.26
 AntiRollAxis: E415."
100.0 7.450580596923828e-07 0.53
 Maltodextrin]
100.0 0.0030517578125 0.21
 paprika
100.0 1.5625 5.45
 lemon powder 3% lemon juice concentrate
100.0 0.01220703125 0.3
 acidity regulators: lemon and vinegar
100.0 1.4901161193847656e-06 1.77
 natural flavor
100.0 0.006103515625 0.31
 sugar
100.0 12.5 3.01
 turmeric
100.0 2.384185791015625e-05 4.67
 tomato puree
100.0 2.9802322387695312e-06 0.23
 salt
0.0 6.25 0.22
 water
0.0 25.0 0.44
 sugar
0.0 12.5 3.01
 acid

 with corn starch (3.3%) and soy protein (3.1%)
0.0 3.3 1.11
 "Sausage meat chicken and turkey
0.0 50 3.73
 water
340.0 25.0 0.44
 salt"
340.0 12.5 0.22
 "Wheat flour
340.0 50 0.65
 strengthening: E 509"
0.0 3.125 2.09
 water
0.0 25.0 0.44
 sugar
0.0 12.5 3.01
 "Apricots 58% (South Africa)
0.0 50 0.23
 mashed apricots 5%
0.0 6.25 4.51
 "Chili-Schoten 20% (red Chili-Schoten
320.0 50 4.51
 colouring: E160c."
320.0 0.048828125 2.41
 salt
320.0 0.09765625 0.22
 water
320.0 12.5 0.44
 acetic acid
320.0 0.390625 1.24
 sugar
320.0 6.25 3.01
 modified tapioca starch
320.0 1.5625 1.11
 red means: Essigsäure)
320.0 3.125 0.12
 acidification: garlic
320.0 0.78125 0.04
 stabilisers: E 415 and E 466
320.0 0.1953125 0.27
 grapes
250.0 6.25 0.83
 peaches
250.0 25.0 0.22
 "Water
250.0 50 0.44
 pears
250.0 12.5 0.34
 peach juice concentrate
250.0 0.78125 1.53
 swine: citric acid"
250.0 0.1953125 1.82
 Kirschhen
250.0 0.390625 3.73
 pear
250.0 1.5625 0.34
 pineapple juice concentrate
250.0 3.125 0.08
 1

0.0 0.1953125 0.26
 split peas
0.0 25.0 0.34
 cinnamon
0.0 0.000762939453125 2.88
 lemon juice concentrate"
0.0 9.5367431640625e-05 1.82
 onions
0.0 0.78125 0.29
 chili
0.0 0.00152587890625 3.43
 sunflower oil
0.0 0.390625 1.48
 cashews
0.0 6.25 9.81
 carrots
0.0 12.5 0.11
 curry powder 2% cumin
0.0 0.09765625 0.3
 ginger
0.0 0.0244140625 0.44
 cornstarch
0.0 0.0003814697265625 0.6
 "6%
0.0 50 0.44
 brown cane sugar
30.0 12.5 3.01
 rapeseed oil
30.0 25.0 0.89
 Popped corn
30.0 50 0.6
 sea salt. May contain unpopped kernels
30.0 6.25 1.14
 apple juice concentrate 5%
0.0 12.5 2.41
 "Applesauce 83%
0.0 50 0.07
 peach aroma
0.0 6.25 0.22
 antioxidants: ascorbic acid
0.0 3.125 0.27
 acidification: citric acid."
0.0 1.5625 1.82
 Peach puree concentrate 12%
0.0 25.0 1.53
 wheat flour
0.0 0.0030517578125 0.3
 salt
0.0 0.00152587890625 0.22
 rapeseed
0.0 0.78125 0.89
 dye: E160c."
0.0 4.76837158203125e-05 0.25
 butter
0.0 0.1953125 1.17
 modified cornstarch
0.0 0.048828125 4.51
 acidity regulat